In [37]:
import cv2
import torch
import requests
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os

In [38]:
video_path = r'./training_set/Videos/'
frame_path = r'./training_set/Frames/'
feature_path = r'./training_set/Features/AlexNetFC7/'

In [39]:
error_log = list()

In [40]:
for filename in os.listdir(feature_path):
    video_name = filename[0:5] #because videos in Features use format xxxxx-xxx.csv
    frame_num = filename[6:9]
    cap = cv2.VideoCapture(video_path + str(int(video_name)) + '.mp4')
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_num) - 1)
    res, frame = cap.read()
    if res == 0:
        error_log.append((video_name, frame_num))
    else:
        cv2.imwrite(frame_path + video_name + '-' + frame_num + '.png', frame)

In [41]:
print(error_log)

[('00165', '186'), ('00261', '179'), ('02152', '172'), ('04461', '165'), ('05657', '129')]
